\
            # 57. 深度学习基础：PyTorch（PyTorch Fundamentals）

            目标：掌握 PyTorch 的核心对象与训练套路：Tensor、autograd、nn.Module、Dataset/DataLoader、训练/评估模式、保存/加载。
本章依赖 `torch`；若未安装会提示安装命令并跳过演示。
提示：Windows 安装 PyTorch 可能需要选择合适的 CUDA/CPU 版本，请以 PyTorch 官方安装指引为准。

            > 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行（第三方依赖会做可选降级）。


## 前置知识

- NumPy 基础（建议）
- 线性代数/导数（了解更佳）
- Python 类/函数基础


## 知识点地图

- 1. PyTorch 训练的最小闭环
- 2. 安装与导入（可选依赖）
- 3. Tensor 基础：shape/dtype/device 与广播
- 4. autograd：requires_grad 与 backward（梯度）
- 5. 定义模型：nn.Module + Linear
- 6. 训练循环（核心）：train/eval、loss、optim
- 7. Dataset/DataLoader：批量训练与 shuffle
- 8. 保存/加载：state_dict（推荐）
- 9. 与 sklearn 的关系（理解）：何时用哪个？


## 自检清单（学完打勾）

- [ ] 理解 Tensor（shape/dtype/device）与基本运算
- [ ] 理解 autograd：requires_grad、backward、梯度累积与清零
- [ ] 会用 nn.Module 定义模型并用 optim 优化
- [ ] 会写最小训练循环（train/eval、loss、step）
- [ ] 会用 Dataset/DataLoader 组织数据与 batch
- [ ] 会保存/加载 state_dict 并复现实验（seed/版本）


In [ ]:
\
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())


## 知识点 1：PyTorch 训练的最小闭环

PyTorch 常见训练流程：
1) 准备数据（Tensor / Dataset / DataLoader）
2) 定义模型（nn.Module）
3) 定义损失（loss）与优化器（optim）
4) 训练循环：forward -> loss -> backward -> step
5) 验证/评估：model.eval() + torch.no_grad()
6) 保存模型：state_dict


## 知识点 2：安装与导入（可选依赖）

安装（CPU 版示例，具体以官方为准）：
- `pip install torch`

如果你需要 GPU/CUDA，请按官方安装命令选择对应版本。


In [ ]:
try:
    import torch
except Exception as e:
    torch = None
    print('torch not available:', type(e).__name__, e)
    print('install: pip install torch')
else:
    print('torch version:', torch.__version__)
    print('cuda available:', torch.cuda.is_available())


## 知识点 3：Tensor 基础：shape/dtype/device 与广播

- shape：张量形状
- dtype：float32/float64/int64...
- device：cpu/cuda

广播规则与 NumPy 类似，但要注意 device 与 dtype 不一致会报错或产生隐式转换。


In [ ]:
try:
    import torch
except Exception:
    print('install torch to run this cell')
else:
    a = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
    b = torch.tensor([10.0, 20.0])
    print('a shape', a.shape, 'dtype', a.dtype, 'device', a.device)
    print('a + b ->\n', a + b)


## 知识点 4：autograd：requires_grad 与 backward（梯度）

- `requires_grad=True`：追踪计算图
- `loss.backward()`：反向传播计算梯度

关键坑：
- 梯度默认会累积（每次 backward 都会累加到 `.grad`），每步需要 `optimizer.zero_grad()`。
- 推理阶段要用 `torch.no_grad()` 避免不必要的图与显存/内存占用。


In [ ]:
try:
    import torch
except Exception:
    print('install torch to run this cell')
else:
    x = torch.tensor([2.0], requires_grad=True)
    y = x * x + 3 * x + 1
    y.backward()
    print('y:', y.item())
    print('dy/dx:', x.grad.item())


## 知识点 5：定义模型：nn.Module + Linear

- nn.Module 是所有模型的基类
- 层（layer）作为成员变量注册
- forward 定义前向计算

下面实现一个最小线性回归模型。


In [ ]:
try:
    import torch
    import torch.nn as nn
except Exception:
    print('install torch to run this cell')
else:
    class LinReg(nn.Module):
        def __init__(self):
            super().__init__()
            self.linear = nn.Linear(1, 1)

        def forward(self, x):
            return self.linear(x)

    m = LinReg()
    x = torch.randn(3, 1)
    y = m(x)
    print('x shape', x.shape, 'y shape', y.shape)


## 知识点 6：训练循环（核心）：train/eval、loss、optim

训练循环的关键步骤：
- `model.train()`：启用训练模式（影响 Dropout/BatchNorm）
- forward：`pred = model(x)`
- loss：`loss = criterion(pred, y)`
- backward：`optimizer.zero_grad(); loss.backward(); optimizer.step()`

评估：
- `model.eval()` + `torch.no_grad()`


In [ ]:
try:
    import torch
    import torch.nn as nn
except Exception:
    print('install torch to run this cell')
else:
    torch.manual_seed(0)

    # synthetic data: y = 3x + 2 + noise
    N = 200
    x = torch.rand(N, 1) * 2 - 1
    y = 3 * x + 2 + 0.1 * torch.randn(N, 1)

    model = nn.Linear(1, 1)
    optim = torch.optim.SGD(model.parameters(), lr=0.1)
    loss_fn = nn.MSELoss()

    for epoch in range(50):
        model.train()
        pred = model(x)
        loss = loss_fn(pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        if (epoch + 1) % 10 == 0:
            w = model.weight.item()
            b = model.bias.item()
            print('epoch', epoch + 1, 'loss', round(loss.item(), 4), 'w', round(w, 3), 'b', round(b, 3))


## 知识点 7：Dataset/DataLoader：批量训练与 shuffle

- Dataset：定义 `__len__` 与 `__getitem__`
- DataLoader：负责 batch、shuffle、多进程加载（num_workers）

注意：Windows 下 num_workers>0 可能需要额外注意（spawn）；入门建议先用 num_workers=0。


In [ ]:
try:
    import torch
    from torch.utils.data import Dataset, DataLoader
    import torch.nn as nn
except Exception:
    print('install torch to run this cell')
else:
    class XY(Dataset):
        def __init__(self, x, y):
            self.x = x
            self.y = y
        def __len__(self):
            return self.x.shape[0]
        def __getitem__(self, idx):
            return self.x[idx], self.y[idx]

    torch.manual_seed(0)
    N = 200
    x = torch.rand(N, 1) * 2 - 1
    y = 3 * x + 2 + 0.1 * torch.randn(N, 1)

    ds = XY(x, y)
    dl = DataLoader(ds, batch_size=32, shuffle=True, num_workers=0)

    model = nn.Linear(1, 1)
    optim = torch.optim.SGD(model.parameters(), lr=0.1)
    loss_fn = nn.MSELoss()

    for epoch in range(10):
        model.train()
        total = 0.0
        for xb, yb in dl:
            pred = model(xb)
            loss = loss_fn(pred, yb)
            optim.zero_grad()
            loss.backward()
            optim.step()
            total += loss.item() * xb.shape[0]
        print('epoch', epoch + 1, 'avg_loss', round(total / len(ds), 4))


## 知识点 8：保存/加载：state_dict（推荐）

推荐保存：
- `torch.save(model.state_dict(), path)`
加载：
- `model.load_state_dict(torch.load(path, map_location='cpu'))`

注意：
- 不建议直接保存整个模型对象（pickle 风险 + 代码变更不兼容）。


In [ ]:
from pathlib import Path

try:
    import torch
    import torch.nn as nn
except Exception:
    print('install torch to run this cell')
else:
    model = nn.Linear(1, 1)
    # fake training result
    with torch.no_grad():
        model.weight.fill_(3.0)
        model.bias.fill_(2.0)

    ART = Path('_nb_artifacts')
    ART.mkdir(exist_ok=True)
    path = ART / 'linreg_state.pt'
    torch.save(model.state_dict(), path)
    print('saved', path)

    m2 = nn.Linear(1, 1)
    m2.load_state_dict(torch.load(path, map_location='cpu'))
    x = torch.tensor([[1.0]])
    print('pred:', m2(x).item())


## 知识点 9：与 sklearn 的关系（理解）：何时用哪个？

- sklearn：传统 ML（线性模型、树模型、聚类等）+ 标准化评估工具链，适合结构化数据 baseline。
- PyTorch：深度学习/自定义模型/自定义训练循环，适合图像/文本/序列与复杂模型。

现实项目常组合：
- 用 pandas/sklearn 做特征与 baseline
- 用 PyTorch 做深度模型或更复杂的学习器


## 常见坑

- 忘记 zero_grad：梯度累积导致训练发散/异常
- 训练时忘记 model.train()/评估时忘记 model.eval()
- 推理没用 torch.no_grad：内存占用变大
- device 不一致：cpu tensor 与 cuda model 混用报错
- shape 不一致：batch 维度丢失导致 Linear 输入错误
- 学习率不合适：太大震荡、太小不收敛


## 综合小案例：实现一个二分类（合成数据）并输出准确率

目标：
- 生成二维点 (x1,x2)，用线性可分规则生成标签
- 用 `nn.Sequential(Linear->Sigmoid)` 或 `BCEWithLogitsLoss` 训练
- 评估准确率

提示：
- 用 `BCEWithLogitsLoss`（更稳定），模型输出 logits，不要手动 Sigmoid。


In [ ]:
try:
    import torch
    import torch.nn as nn
except Exception:
    print('install torch to run this cell')
else:
    torch.manual_seed(0)
    N = 400
    X = torch.randn(N, 2)
    # linear rule: x1 + x2 > 0 -> 1 else 0
    y = (X[:, 0] + X[:, 1] > 0).float().unsqueeze(1)

    model = nn.Linear(2, 1)
    loss_fn = nn.BCEWithLogitsLoss()
    optim = torch.optim.SGD(model.parameters(), lr=0.2)

    for epoch in range(50):
        logits = model(X)
        loss = loss_fn(logits, y)
        optim.zero_grad()
        loss.backward()
        optim.step()

    with torch.no_grad():
        pred = (torch.sigmoid(model(X)) > 0.5).float()
        acc = (pred.eq(y)).float().mean().item()
    print('acc:', round(acc, 4))


## 自测题（不写代码也能回答）

- autograd 的作用是什么？为什么梯度会累积？
- model.train() 与 model.eval() 的差异是什么？
- 为什么推理阶段要用 torch.no_grad()？
- 为什么推荐保存 state_dict 而不是保存整个模型对象？


## 练习题（建议写代码）

- 把 mini_case 改成：训练/验证拆分，并输出验证集准确率。
- 给训练循环加入学习率调度（StepLR，了解）。
- 用 FastAPI 写一个 /predict 接口加载 state_dict 做推理（与框架章节呼应）。
